# Soft Prompt Tuning

Prompt tuning adds a small set of trainable virtual tokens (soft prompts) to the input while keeping the pre-trained model's weight frozen.
These virtual prompts are not human-readable; they are appended to the start of a prompt to serve as a task-specific guide during LLM training or inferences.

For example, the virtual tokens are prefixed to text for sentiment classification:

```
[virtual tokens] I love Fridays!
```

where `[virtual tokens]` are the inserted embeddings. These virtual tokens can be randomly generated or initialized from a vocabulary.

During training, these token embeddings are updated while the base model remains frozen.

In [1]:
# Run the following to downgrade datasets module, restart kernel
!pip install datasets==3.6.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [1]:
# Import packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, PeftConfig, PeftModel
from datasets import load_dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

In [2]:
model_name = "bigscience/bloomz-560m"

username = 'ought/raft'
dataset_name = "twitter_complaints"

In [3]:
# TODO: Load dataset
dataset = load_dataset(username, dataset_name)


README.md: 0.00B [00:00, ?B/s]

raft.py: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/266k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [8]:
# TODO: Display dataset
#print(dataset)
idx = 10
print(dataset['train'][idx])
print(dataset['train'].features['Label'].names)


{'Tweet text': '@NortonSupport Thanks much.', 'ID': 10, 'Label': 2}
['Unlabeled', 'complaint', 'no complaint']


In [9]:
# TODO: Add text_label from Label
classes = dataset['train'].features['Label'].names
print(classes)

['Unlabeled', 'complaint', 'no complaint']


In [10]:
# Convert labels to label text
dataset_enh = dataset.map(
   lambda x: { 'text_label': [ classes[label] for label in x['Label'] ] },
   batched = True,
   num_proc = 1
)

print(dataset_enh['train'][2])

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

{'Tweet text': "If I can't get my 3rd pair of @beatsbydre powerbeats to work today I'm doneski man. This is a slap in my balls. Your next @Bose @BoseService", 'ID': 2, 'Label': 1, 'text_label': 'complaint'}


In [11]:
# TODO: Load tokenizer
# call variable name tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [12]:
# Utility function to preprocess teweets

def preprocess_function(examples):
    text_column = 'Tweet text'
    label_column = 'text_label'
    max_length = 64
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
# TODO: Preprocess the Tweets
# Call preprocessed tweets dataset_processed
dataset_processed = dataset_enh.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=False
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [16]:
# TODO: Explore preprocessed Tweets
print(dataset_processed['train'][10])
dec_text = tokenizer.decode(dataset_processed['train'][10]['input_ids'])
print(dec_text)


{'Tweet text': '@NortonSupport Thanks much.', 'ID': 10, 'Label': 2, 'text_label': 'no complaint', 'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 227985, 5484, 915, 2566, 49, 656, 266, 53312, 44639, 4936, 17, 77658, 915, 210, 1936, 106863, 3], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1936, 106863, 3]}
<pad><pad><pad><pad><pad><pad><pad><pad><p

In [17]:
from huggingface_hub import load_state_dict_from_file
# TODO: Create train and evaluation dataset
# Must call train set DataLoader loader_train
# Must call test set DataLoader loader_test

batch_size = 8

dataset_train = dataset_processed['train']
dataset_test = dataset_processed['test']

loader_train = DataLoader(dataset_train, collate_fn=default_data_collator, batch_size=batch_size)
loader_test = DataLoader(dataset_test, collate_fn=default_data_collator, batch_size=batch_size)



## Soft Prompt

In [25]:
# TODO: Create PEFT configuration
config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=8,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    tokenizer_name_or_path=model_name
)

In [26]:
# TODO: Create model for training
base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = get_peft_model(base_model, peft_config=config)


Loading weights:   0%|          | 0/293 [00:00<?, ?it/s]

In [27]:
model.print_trainable_parameters()

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0015


In [ ]:
# Create optimizer
lr = 3e-2
num_epochs = 1
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
   optimizer=optimizer,
   num_warmup_steps=0,
   num_training_steps=(len(loader_train) * num_epochs)
)

In [ ]:
# Train model
# default to CPU
device = "cuda"
device = "cpu"

model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(loader_train)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(loader_test)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(loader_test)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(loader_train)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

## Inferencing

In [33]:
peft_model_id = "difince/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

In [34]:
## TODO: Load trained model
base_model = AutoModelForCausalLM.from_pretrained(model_name)
vt_model = PeftModel.from_pretrained(base_model, peft_model_id)


Loading weights:   0%|          | 0/293 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

In [41]:
## TODO: Write and encode text

input_text = "Tweet text: @uni I hate the food in your canteen. Label: "
input_text = "Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label: "

input_enc = tokenizer(input_text, return_tensors='pt')
print(input_enc)



{'input_ids': tensor([[227985, 106659,   2566,  71187,  29069,   1387,  55907,   1809,   3784,
            578,  17444,    613,   3478,   2592,  13802,     17,   6728,   8256,
           1119, 227284,     34,  77658,     29,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [42]:
## TODO: Determine sentiment from model
result_enc = vt_model.generate(
    input_ids = input_enc['input_ids'],
    attention_mask = input_enc['attention_mask'],
    eos_token_id=3
)

print(result_enc)

print(tokenizer.decode(result_enc[0], skip_special_tokens=True))



tensor([[227985, 106659,   2566,  71187,  29069,   1387,  55907,   1809,   3784,
            578,  17444,    613,   3478,   2592,  13802,     17,   6728,   8256,
           1119, 227284,     34,  77658,     29,    210,   1936, 106863,      3]])
Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label: no complaint
